In [4]:
import pandas as pd
import numpy as np

data_path = "../../data/"
save_path = "../data/"

tagged_tracks = pd.read_csv(data_path + "tracks_tagged_v5.csv")
linked_tracks = pd.read_csv(data_path + "tracks_radar_linked_filtered.csv")
ais_tracks = pd.read_csv(data_path + "tracks_ais.csv")

In [5]:
labeled_tagged_tracks = tagged_tracks[tagged_tracks["activity"].notna()]
labeled_tagged_tracks["activity"].value_counts()
labeled_tagged_tracks[["id_track", "activity"]].to_csv(save_path + "activity_label.csv", index=False)

In [10]:
# First need to perform type mapping on raw ais data
class VesselTypeAggregator:
    """
    Aggregate vessel type based on defined categories from insights
    """
    
    def __init__(self, ais_df):
        self.type_mapping = {
            'tanker_ship': 'cargo/tanker',
            'cargo_ship': 'cargo/tanker',
            'tug': 'tug/tow',
            'towing_ship': 'tug/tow',
            'fishing_boat': 'fishing_boat',
            'commercial_fishing_boat': 'fishing_boat',
            'military_ship': 'military_ship',
            'class_b':'class_b',
            'passenger_ship': 'passenger_ship',
            'pleasure_craft': 'pleasure_craft',
            'sailboat': 'pleasure_craft',
            'search_and_rescue_boat': 'other',
            'pilot_boat': 'other',
            'high_speed_craft': 'other',
            'law_enforcement_boat': 'other',
            'other': 'other',
            'unknown': 'other'
        }
        self.df = ais_df.copy()
        
    def apply(self):
        self.df['type_m2_agg'] = self.df['type_m2'].map(self.type_mapping).fillna('other')
    
    def get_result(self):
        return self.df[["id_track", "type_m2", "type_m2_agg"]]

VTA = VesselTypeAggregator(ais_tracks)
VTA.apply()
ais_labels = VTA.get_result()

In [11]:
def fetch_ais_type(ais_id):
    find_match = ais_labels[ais_labels["id_track"] == ais_id]
    if len(find_match) == 0:
        raise RuntimeError(f"Track id {ais_id} does not have associated records")
    type = find_match.iloc[0]["type_m2_agg"]
    return type

labeled_radar_tracks = linked_tracks[["id_track"]].copy()
labeled_radar_tracks["type_m2_agg"] = linked_tracks.apply(lambda row: fetch_ais_type(row["ais_id"]), axis = 1)

In [18]:
labeled_radar_tracks["type_m2_agg"].value_counts()
labeled_radar_tracks.to_csv(save_path + "type_label.csv", index=False)